LUCAS KANADE METHOD

In [8]:
import numpy as np 
import cv2

In [9]:
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3,minDistance=7, blockSize=7)

In [10]:
lk_params = dict(winSize=(200,200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TermCriteria_COUNT,10,0.03)) #windowun küçük olursa kaçırma ihtimalin yüksek, noise'den etkileniyor.
#Max level da resolution'ı değiştirip orada arıyor optical flowu 

In [12]:
cap = cv2.VideoCapture(0) #livestream oluyor

ret, prev_frame = cap.read() #ilk alınan frame

prev_gray= cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

#pts to track
prevPts= cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

mask =np.zeros_like(prev_frame) #line çizmek için.

while True:
    ret,frame = cap.read()

    frame_gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    nextPts ,status, err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params) #default bunlar dönüyor.
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]

    for i, (new,prev) in enumerate (zip(good_new,good_prev)):
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        x_new, y_new = int(x_new), int(y_new)
        x_prev, y_prev = int(x_prev), int(y_prev)
        mask=cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)

        frame = cv2.circle(frame,(x_new,y_new),8,(255,0,0),-1)
    img = cv2.add(frame,mask)
    cv2.imshow('tracking',img)

    k= cv2.waitKey(30) & 0xFF
    if k == 27:
        break

    prev_gray = frame_gray.copy()
    prevPts =good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()
